In [25]:
import os

os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/kirankarakalli/Predicting-Wine-Quality-Using-ML.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="kirankarakalli"
os.environ["MLFLOW_TRACKING_PASSWORD"]="87b671153c6abc5700206e41888a56e7d6cb0e49"

In [2]:
%pwd

'c:\\Users\\karakkir\\OneDrive - Tietoevry\\Predicting-Wine-Quality-Using-ML\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\karakkir\\OneDrive - Tietoevry\\Predicting-Wine-Quality-Using-ML'

In [5]:
from dataclasses import dataclass
from pathlib import Path

In [26]:

@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str


In [27]:
from src.datascience.utils.common import read_yaml,create_directories,save_json
from src.datascience.constants import *

In [28]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config=self.config.model_evaluation
        params=self.params.ElasticNet
        schema=self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config=ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            mlflow_uri="https://dagshub.com/kirankarakalli/Predicting-Wine-Quality-Using-ML.mlflow"


        )
        return model_evaluation_config


In [29]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [34]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    
    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]

        mlflow.set_tracking_uri(self.config.mlflow_uri)

        with mlflow.start_run():

            predicted_qualities = model.predict(test_x)

            (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)

            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)

            joblib.dump(model, "model.pkl")
            mlflow.log_artifact("model.pkl")


In [35]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2025-12-03 20:05:03,057- INFO- common - yaml file :config\config.yaml loaded successfully]
[2025-12-03 20:05:03,061- INFO- common - yaml file :params.yaml loaded successfully]
[2025-12-03 20:05:03,065- INFO- common - yaml file :schema.yaml loaded successfully]
[2025-12-03 20:05:03,070- INFO- common - created directory at:artifacts]
[2025-12-03 20:05:03,072- INFO- common - created directory at:artifacts/model_evaluation]
[2025-12-03 20:05:04,019- INFO- common - json file saved at:artifacts\model_evaluation\metric.json]
🏃 View run grandiose-shark-896 at: https://dagshub.com/kirankarakalli/Predicting-Wine-Quality-Using-ML.mlflow/#/experiments/0/runs/1c799fe602e34c889fc6148d3cfc32c0
🧪 View experiment at: https://dagshub.com/kirankarakalli/Predicting-Wine-Quality-Using-ML.mlflow/#/experiments/0
